# o-viton
## shape generation
### train

#### 필요한 패키지 설치

In [1]:
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import sys

IS_COLAB = False

if 'COLAB_GPU' in os.environ:
    IS_COLAB=True
else:
    IS_COLAB=False
 

In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install dominate
!pip install av
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# import torch
# assert torch.__version__.startswith("1.7")
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html


In [5]:
# console에서 다음 작업 진행할 것.

#!git clone https://github.com/wonyangcho/detectron2.git
# !cd detectron2
# !git checkout develop
# !git pull
# !cd ..
!git clone https://github.com/wonyangcho/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data.git o-viton
# !cd o-viton
# !git checkout develop
# !git pull
# !cd ..

fatal: destination path 'o-viton' already exists and is not an empty directory.


In [6]:
if IS_COLAB == True:
    from google.colab import drive


In [7]:
!nvidia-smi

Fri Feb  5 12:19:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    32W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### 데이터 불러오기

In [8]:
from pathlib import Path

my_file = Path("/content/dataset")
if my_file.is_dir() == False:
  !wget -O o-viton-dataset.zip https://www.dropbox.com/s/b5ravd3x47sw5tx/o-viton-dataset_train.zip?dl=0
  #!wget -O o-viton-dataset.zip https://www.dropbox.com/s/82jngyj52qy48mg/dataset_train_org.zip?dl=0
  !unzip -o -q o-viton-dataset.zip 
  !rm o-viton-dataset.zip
!ls ./dataset


train_densepose  train_img  train_label


#### google drive 연결

In [9]:
if IS_COLAB == True:
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/02_o-viton/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
source


#### path 설정

In [10]:
if IS_COLAB == True:
  sys.path.append("/content/drive/My Drive/02_o-viton/source/shape_generation")
  sys.path.append("/content/o-viton/shape_generation")

#### 관련 모듈 불러오기 

In [11]:
from options.train_options import TrainOptions
from util.visualizer import Visualizer
import util.util as util
from models.models import create_model
from data.ov_train_dataset import RegularDataset
import time
import os
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
from collections import OrderedDict
from subprocess import call
import fractions
from torch.utils.tensorboard import SummaryWriter
from multiprocessing import Process
import matplotlib

%matplotlib inline

#### checkpoint 경로 확인

In [12]:
!ls /content/drive/MyDrive/02_o-viton/source/shape_generation/checkpoints

vfr


#### main

In [13]:
def lcm(a, b): return abs(a * b)/fractions.gcd(a, b) if a and b else 0

def main():
    opt = TrainOptions().parse(save=False)
    opt.checkpoints_dir = "/content/drive/My Drive/02_o-viton/source/shape_generation/checkpoints"
    opt.dataroot = "./dataset"
    opt.continue_train = True #이전 checkpoint부터 학습을 하고 싶으면 True.
    opt.gpu_ids = "0"
    opt.niter = 100
    opt.niter_decay = 0
    opt.batchSize = 4

    opt.print_freq = lcm(opt.print_freq, opt.batchSize)
    if opt.debug:
        opt.display_freq = 1
        opt.print_freq = 1
        opt.niter = 1
        opt.niter_decay = 0
        opt.max_dataset_size = 10
        opt.save_latest_freq = 1 #잠시만
        
    
   
    iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')
    if opt.continue_train:
        try:
            start_epoch, epoch_iter = np.loadtxt(
                iter_path, delimiter=',', dtype=int)
        except:
            start_epoch, epoch_iter = 1, 0
        print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))
    else:
        start_epoch, epoch_iter = 1, 0

   

    # NEW DATALOADER
    augment = {}

   
    augment['1'] = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.RandomAffine(degrees=10, translate=(
                0.1, 0.1), scale=(0.8, 1.2), shear=20),
            transforms.ToTensor()])  # change to [C, H, W]

    augment['2'] = transforms.Compose(
        [
            transforms.ToTensor()])  # change to [C, H, W]

    train_dataset = RegularDataset(opt, augment)

    train_dataloader = DataLoader(train_dataset,
                                batch_size=opt.batchSize,
                                shuffle=True,
                                num_workers=int(opt.nThreads),
                                pin_memory=True)
  


    dataset_size = len(train_dataset)
    print('#training images = %d' % dataset_size)

    # FOR DEBUGGING
    print(" #Checking  the dimension and type of data")
    for key in train_dataset[0].keys():
        try:
            x = train_dataset[0][key]
            print("name of the input and shape -- > ", key, x.shape)
            print("type,dtype,and min max -- >", type(x),
                x.dtype, torch.min(x), torch.max(x))
        except Exception as e:
            print("name of the input -- > ", key, train_dataset[0][key])
        print('----------------')

    dataset_size = len(train_dataset)
    print('#training images = %d' % dataset_size)

    # Initialize Networks
    model = create_model(opt)

    # Training Visualizer
    visualizer = Visualizer(opt)

    # Optimizers
    optimizer_G, optimizer_D = model.module.optimizer_G, model.module.optimizer_D

    # Train related additional details
    total_steps = (start_epoch-1) * dataset_size + epoch_iter
    display_delta = total_steps % opt.display_freq
    print_delta = total_steps % opt.print_freq
    save_delta = total_steps % opt.save_latest_freq


    for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
        epoch_start_time = time.time()
        if epoch != start_epoch:
            epoch_iter = epoch_iter % dataset_size
        for i, data in enumerate(train_dataloader, start=epoch_iter):
            if total_steps % opt.print_freq == print_delta:
                iter_start_time = time.time()
            total_steps += opt.batchSize
            epoch_iter += opt.batchSize

            # whether to collect output images
            save_fake = total_steps % opt.display_freq == display_delta

            ############## Forward Pass ######################
            # Reference Purpose
            # input_dict = {'seg_map': A_tensor, 'dense_map': dense_img, 'target': B_tensor, 'seg_map_path': A_path,
            # 'target_path': A_path, 'densepose_path': dense_path }
            # print( data['seg_mask'].shape)
            losses, generated = model(
                data['seg_map'], data['dense_map'], data['target'], data['seg_mask'], infer=save_fake)

            # sum per device losses
            losses = [torch.mean(x) if not isinstance(x, int)
                    else x for x in losses]
            loss_dict = dict(zip(model.module.loss_names, losses))

            # calculate final loss scalar
            loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
            loss_G = loss_dict['G_GAN'] + loss_dict.get('G_CE', 0)

            ############### Backward Pass ####################
            # update generator weights
            optimizer_G.zero_grad()
            loss_G.backward()
            optimizer_G.step()

            # update discriminator weights
            optimizer_D.zero_grad()
            loss_D.backward()
            optimizer_D.step()

            ############## Display results and errors ##########
            # print out errors
            if total_steps % opt.print_freq == print_delta:
                errors = {k: v.data.item() if not isinstance(
                    v, int) else v for k, v in loss_dict.items()}
                t = (time.time() - iter_start_time) / opt.print_freq
                visualizer.print_current_errors(epoch, epoch_iter, errors, t)
                visualizer.plot_current_errors(errors, total_steps)
                #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"])

            # display output images
            if save_fake:
                visuals = OrderedDict([('input_label', util.tensor2label(data['seg_map'][0], opt.label_nc)),
                                    ('synthesized_image', util.tensor2label(
                                        generated.data[0], opt.label_nc)),
                                    ('real_image', util.tensor2label(data['target'][0], opt.label_nc))])
                visualizer.display_current_results(visuals, epoch, total_steps)

            # save latest model
            if total_steps % opt.save_latest_freq == save_delta:
                print('saving the latest model (epoch %d, total_steps %d)' %
                    (epoch, total_steps))
                model.module.save('latest')
                np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

            if epoch_iter >= dataset_size:
                break

        # end of epoch
        iter_end_time = time.time()
        print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

        # save model for this epoch
        if epoch % opt.save_epoch_freq == 0:
            print('saving the model at the end of epoch %d, iters %d' %
                (epoch, total_steps))
            model.module.save('latest')
            model.module.save(epoch)
            np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

        # linearly decay learning rate after certain iterations
        if epoch > opt.niter:
            model.module.update_learning_rate()
        

if __name__ == '__main__':
    # torch.multiprocessing.set_start_method('spawn')
    main()

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: /home/dataset/
debug: False
densepose_nc: 27
display_freq: 100
display_winsize: 512
feat_num: 10
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 20
instance_feat: False
isTrain: True
label_feat: False
label_nc: 20
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: vfr
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_ce_loss: False
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 20
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 5
save_latest_freq: 1000
se

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: fractions.gcd() is deprecated. Use math.gcd() instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional.py:92: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


name of the input and shape -- >  seg_map torch.Size([20, 512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(0.) tensor(1.)
----------------
name of the input and shape -- >  dense_map torch.Size([27, 512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(0.) tensor(1.)
----------------
name of the input and shape -- >  target torch.Size([20, 512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(0.) tensor(1.)
----------------
name of the input -- >  seg_map_path ./dataset/train_label/000003_0.png
----------------
name of the input -- >  target_path ./dataset/train_label/000003_0.png
----------------
name of the input -- >  densepose_path ./dataset/train_densepose/000003_0.npy
----------------
name of the input and shape -- >  seg_mask torch.Size([512, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.int64 tensor(0) tensor(15)
----------------
#training images = 14220
--------- model 